In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import ui

## Test objects

In [3]:
point = ee.Geometry.Point([-72, -42])
image = ee.Image.constant(0)
date = ee.Date('2000-1-1')

## `getInfo`
Get information asynchronously. It always returns a `dict` with one element: `info`. Inside this element will be the result of the object. The result is a modified `dict`

In [4]:
infop = ui.getInfo(point)

In [5]:
infop.get()

In [6]:
infop()

{'coordinates': [-72.0, -42.0], 'type': 'Point'}

In [7]:
info_date = ui.getInfo(date)

In [9]:
info_date()

{'type': 'Date', 'value': 946684800000}

## `eprint`
Print Earth Engine object

In [10]:
ui.eprint(point, notebook=True, do_async=True)

In [11]:
ui.eprint(point, date, notebook=True, do_async=True)

In [12]:
ui.eprint(image, date, notebook=False, do_async=True)

Cannot make async printing outside a Jupyter environment
{ 'bands': [ { 'crs': 'EPSG:4326',
               'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
               'data_type': { 'max': 0,
                              'min': 0,
                              'precision': 'int',
                              'type': 'PixelType'},
               'id': 'constant'}],
  'type': 'Image'}

'2000-01-01T00:00:00'

